<a href="https://colab.research.google.com/github/pythonuzgit/elmurodov/blob/master/Image_classification_model_using_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"uzazam","key":"7abb2778b77e603b3a87069f481fc6a6"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#change the permission

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d aadityasinghal/facial-expression-dataset

 96% 58.0M/60.7M [00:00<00:00, 69.0MB/s]
100% 60.7M/60.7M [00:00<00:00, 104MB/s] 


In [4]:
from zipfile import ZipFile
file_name = "/content/facial-expression-dataset.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [6]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
print(dev)

cuda


Transforms

In [8]:
transformer = transforms.Compose([
                                  transforms.Resize((150, 150)),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.5, 0.5, 0.5],
                                                       [0.5, 0.5, 0.5])
])

DataLoader

In [9]:
#Path for trainig and testing directory

train_path = '/content/train/train'
test_path = '/content/test/test'

train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform = transformer),
    batch_size = 256, shuffle = True
)

test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform = transformer),
    batch_size = 256, shuffle = True
)

categories

In [10]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1]
                  for j in root.iterdir()])

print(classes)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


CNN Network

In [11]:
class ConvNet(nn.Module):
  def __init__(self, num_classes = 6):
    super(ConvNet, self).__init__()

    #Output size after convolutional filter
    #((w - f + 2*P)/ s) + 1

    #input shape = (256, 3, 150, 150)

    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 12, kernel_size = 3,
                           stride = 1, padding = 1)
    
    #Shape = (256, 12, 150, 150)

    self.bn1 = nn.BatchNorm2d(num_features = 12)

    #Shape = (256, 12, 150, 150)

    self.relu1 = nn.ReLU()

    #Shape = (256, 12, 150, 150)

    self.pool = nn.MaxPool2d(kernel_size = 2)

    #Reduce the image size be factor 2
    #Shape = (256, 12, 75, 75)

    self.conv2 = nn.Conv2d(in_channels = 12, out_channels = 20, kernel_size = 3,
                           stride = 1, padding = 1)
    
    #Shape = (256, 20, 150, 150)

    self.relu2 = nn.ReLU()

    #Shape = (256, 20, 75, 75)

    self.conv3 = nn.Conv2d(in_channels = 20, out_channels = 32,
                           kernel_size = 3, stride = 1, padding = 1)

    #Shape = (256, 32, 75, 75)

    self.bn3 = nn.BatchNorm2d(num_features = 32)

    #Shape = (256, 32, 75, 75)

    self.relu3 = nn.ReLU()

    #Shape = (256, 32, 75, 75)

    self.fc = nn.Linear(in_features = 32*75*75, out_features = num_classes)

  def forward(self, input):
    output = self.conv1(input)
    output = self.bn1(output)
    output = self.relu1(output)

    output = self.pool(output)

    output = self.conv2(output)
    output = self.relu2(output)

    output = self.conv3(output)
    output = self.bn3(output)
    output = self.relu3(output)

    output = output.view(-1, 32 * 75 * 75)

    output = self.fc(output)
    return output

In [12]:
model = ConvNet(num_classes = 7)
model = model.cuda()

Optimizer and loss function

In [13]:
optimizer = Adam(model.parameters(), lr = 0.001, weight_decay = 0.0001)

loss_function = nn.CrossEntropyLoss()

In [14]:
num_epochs = 20

Caluclating the size of training and testing iages

In [15]:
train_count = len(glob.glob(train_path + '/**/*.jpg'))
test_count = len(glob.glob(test_path + '/**/*.jpg'))

In [16]:
print(train_count, test_count)

28709 7178


Model training and svaing best model

In [17]:
best_accuracy = 0.0

for epoch in range(num_epochs):
  model.train()
  train_accuracy = 0.0
  train_loss = 0.0

  for i, (images, labels) in enumerate(train_loader):
    if torch.cuda.is_available:
      images = Variable(images.cuda())
      labels = Variable(labels.cuda())

    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.cpu().data * images.size(0)
    _, prediction = torch.max(outputs.data, 1)
    train_accuracy += int(torch.sum(prediction == labels.data))

  train_accuracy = train_accuracy/train_count
  train_loss = train_loss/train_count

  model.eval()

  test_accuracy = 0.0

  for i, (images, labels) in enumerate(test_loader):
    if torch.cuda.is_available:
      images = Variable(images.cuda())
      labels = Variable(labels.cuda())

    outputs = model(images)
    _, prediction = torch.max(outputs.data, 1)
    test_accuracy += int(torch.sum(prediction == labels.data))


  test_accuracy = test_accuracy/test_count


  print('Epoch: '+str(epoch)+' Train loss: '+str(int(train_loss))+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))


  #Save the best model

  if test_accuracy>best_accuracy:
    torch.save(model.state_dict(), 'best_checkpoint.model')
    best_accuracy = test_accuracy    


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0 Train loss: 11 Train Accuracy: 0.2686265630986799 Test Accuracy: 0.23808860406798552
Epoch: 1 Train loss: 1 Train Accuracy: 0.37078964784562335 Test Accuracy: 0.35135135135135137
Epoch: 2 Train loss: 1 Train Accuracy: 0.4352642028632136 Test Accuracy: 0.36040679855112845
Epoch: 3 Train loss: 1 Train Accuracy: 0.4802675119300568 Test Accuracy: 0.404151574254667
Epoch: 4 Train loss: 1 Train Accuracy: 0.523076387195653 Test Accuracy: 0.387712454722764
Epoch: 5 Train loss: 1 Train Accuracy: 0.5460656936849072 Test Accuracy: 0.44524937308442464
Epoch: 6 Train loss: 1 Train Accuracy: 0.5838587202619387 Test Accuracy: 0.42323767066035106
Epoch: 7 Train loss: 1 Train Accuracy: 0.6226618830331951 Test Accuracy: 0.36388966285873503
Epoch: 8 Train loss: 0 Train Accuracy: 0.6429691037653698 Test Accuracy: 0.3959320144887155
Epoch: 9 Train loss: 0 Train Accuracy: 0.6738305061130656 Test Accuracy: 0.4604346614655893
Epoch: 10 Train loss: 0 Train Accuracy: 0.717754014420565 Test Accuracy: 0.